# PyTorchVision을 참고하여 Pre-Trained AlexNet/VGG/ResNet/DenseNet 성능 비교하기

In [2]:
import torch
import torchvision
import torchvision.transforms as T

In [40]:
transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root="./data", train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root="./data", train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

CLASSES = ('plane', 'car', 'bird', 'cat', 'deer',
            'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import numpy as np
import matplotlib.pyplot as plt

def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [57]:
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights, alexnet, AlexNet_Weights, vgg16, VGG16_Weights, densenet121, DenseNet121_Weights

alexNet = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
for param in alexNet.parameters():
    param.requires_grad = (False)
alexNet.classifier[-1] = nn.Linear(4096, 10)

vgg = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
for param in vgg16.parameters():
    param.requires_grad = (False)
vgg.classifier[-1] = nn.Linear(4096, 10)

resNet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
for param in resNet.parameters():
    param.requires_grad = (False)
in_features = resNet.fc.in_features
resNet.fc = nn.Linear(in_features, len(CLASSES))

denseNet = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
for param in densenet121.parameters():
    param.requires_grad = (False)
in_features = denseNet.classifier.in_features
denseNet.classifier = nn.Linear(in_features, len(CLASSES))

models = {
    # 'alexNet': alexNet,
    'vgg': vgg,
    'resNet': resNet,
    'denseNet': denseNet
}

In [58]:
import torch.optim as optim


def train_predict(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), 0.0001, momentum=0.9)

    # Train the net
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    for epoch in range(1):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:
                print(f"[{epoch + 1}, {i + 1:5d} loss: {running_loss / 2000:.3f}]")
                running_loss = 0.0

    print("Finished Training")

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predictions == labels).sum().item()

    print(f"Accuracy of the net on the 10000 test images: {100 * correct / total} %")

In [59]:
for name, model in models.items():
    print(name)
    train_predict(model)

vgg
[1,  2000 loss: 1.119]
[1,  4000 loss: 0.763]
[1,  6000 loss: 0.659]
[1,  8000 loss: 0.587]
[1, 10000 loss: 0.581]
[1, 12000 loss: 0.541]
Finished Training
Accuracy of the net on the 10000 test images: 83.56 %
resNet
[1,  2000 loss: 2.091]
[1,  4000 loss: 1.876]
[1,  6000 loss: 1.750]
[1,  8000 loss: 1.651]
[1, 10000 loss: 1.580]
[1, 12000 loss: 1.528]
Finished Training
Accuracy of the net on the 10000 test images: 47.93 %
denseNet
[1,  2000 loss: 1.975]
[1,  4000 loss: 1.711]
[1,  6000 loss: 1.598]
[1,  8000 loss: 1.515]
[1, 10000 loss: 1.439]
[1, 12000 loss: 1.368]
Finished Training
Accuracy of the net on the 10000 test images: 54.76 %
